Question 2: What is the correlation between the number of times a stock is discussed on the WSJ and its stock price? (could also do sentiment analysis)
- Data from: https://www.kaggle.com/datasets/marta99/elon-musks-tweets-dataset-2022?resource=download

In [15]:
import pandas as pd
import plotly.express as px 
from scipy.stats import pearsonr, chi2_contingency
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer



In [25]:
sentence_analyzer = SentimentIntensityAnalyzer()
temp_sentence = "I don't mind Donald Trump"
sentiment_result = sentence_analyzer.polarity_scores(temp_sentence)

print(sentiment_result['neu'])

# combine all the info to a particular day

1.0
